In [19]:
import dgl
import utils
import base
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph
import numpy as np
import pandas as pd
import random

from dgl.nn.pytorch import RelGraphConv
from torch.autograd import Variable

ModuleNotFoundError: No module named 'base'

#### Relational Graph Covo Networks (R-GCN)

In [3]:
class RGCN(nn.Module):
    def __init__(
        self,
        num_nodes,
        h_dim,
        out_dim,
        num_rels,
        regularizer="basis",
        num_bases=-1,
        dropout=0.0,
        self_loop=False,
        ns_mode=False,
    ):
        super(RGCN, self).__init__()

        if num_bases == -1:
            num_bases = num_rels
        self.emb = nn.Embedding(num_nodes, h_dim)
        self.conv1 = RelGraphConv(
            h_dim, h_dim, num_rels, regularizer, num_bases, self_loop=self_loop
        )
        self.conv2 = RelGraphConv(
            h_dim,
            out_dim,
            num_rels,
            regularizer,
            num_bases,
            self_loop=self_loop,
        )
        self.dropout = nn.Dropout(dropout)
        self.ns_mode = ns_mode

    def forward(self, g, nids=None):
        if self.ns_mode:
            # forward for neighbor sampling
            x = self.emb(g[0].srcdata[dgl.NID])
            h = self.conv1(g[0], x, g[0].edata[dgl.ETYPE], g[0].edata["norm"])
            h = self.dropout(F.relu(h))
            h = self.conv2(g[1], h, g[1].edata[dgl.ETYPE], g[1].edata["norm"])
            return h
        else:
            x = self.emb.weight if nids is None else self.emb(nids)
            h = self.conv1(g, x, g.edata[dgl.ETYPE], g.edata["norm"])
            h = self.dropout(F.relu(h))
            h = self.conv2(g, h, g.edata[dgl.ETYPE], g.edata["norm"])
            return h


In [20]:
# Define model parameters
num_nodes =  # Specify the number of nodes
h_dim =  # Specify the input feature dimension
out_dim =  # Specify the output dimension
num_rels = 5 # Specify the number of relation types in your graph
regularizer = "basis"  # You can change this based on your requirements
num_bases = -1  # You can change this based on your requirements
dropout = 0.0  # You can change this based on your requirements
self_loop = False  # You can change this based on your requirements
ns_mode = False  # You can change this based on your requirements

# Instantiate the RGCN model
model = RGCN(
    num_nodes=num_nodes,
    h_dim=h_dim,
    out_dim=out_dim,
    num_rels=num_rels,
    regularizer=regularizer,
    num_bases=num_bases,
    dropout=dropout,
    self_loop=self_loop,
    ns_mode=ns_mode,
)

SyntaxError: invalid syntax (2036066188.py, line 2)

#### Model Train 1 - Assign Paper

In [5]:
## Evaluation Metrics
def hetro_loss(x, mu, v):
    return (((x - mu) ** 2 / v) + torch.log(v)).mean()

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = y_true.cpu().numpy(), y_pred.cpu().numpy()
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def compute_rmse(y_true, y_pred):
    y_true, y_pred = y_true.cpu().numpy(), y_pred.cpu().numpy()
    return np.sqrt(((y_true/60 - y_pred/60) ** 2).mean())

In [6]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [7]:
def train(model,optimizer,epoch,batch_size, train_data):

    train_path, train_ratio, train_slots, train_orderid, train_label =train_data
    road_idx=torch.tensor(list(range(num_locations))).to(device)
    time_idx=torch.tensor(list(range(num_times))).to(device)       
    model.train()
    tte_loss,speed_loss=0.0,0.0,0.0,0.0,0.0
    for i in range(0, train_path.shape[0],batch_size):
        paths_temp,ratio_temp,time_temp=train_path[i:i+batch_size],train_ratio[i:i+batch_size],train_slots[i:i+batch_size]
        optimizer.zero_grad()
        entire_out,mu,sigma=model(road_idx,time_idx,time_temp,g2,u,lengths,paths_temp,edge_type)
            # add 1
        
        mu=torch.cat([mu,torch.zeros(1,mu.shape[1]).to(device)])
        sigma=torch.cat([sigma,torch.zeros(1,sigma.shape[1]).to(device)])
        t=train_slots[i:i+batch_size]
        
        path_mu,path_sigma=mu[paths_temp],sigma[paths_temp]
        path_mu_new,path_mu_path_sigma=[],[]
        for o,l in enumerate(t):
            path_mu_new.append(path_mu[o:o+1,:,l])
            path_mu_path_sigma.append(path_sigma[o:o+1,:,l])
        path_mu,path_sigma=torch.cat(path_mu_new),torch.cat(path_mu_path_sigma)
    
        E_mu=torch.mul(path_mu,ratio_temp).sum(dim=1)+eps
        E_sigma=torch.mul(path_sigma,ratio_temp).pow(2).sum(dim=1)+eps

    
        label_time=train_label[i:i+batch_size,0:1]
        label_speed=train_label[i:i+batch_size,1]
        loss1=hetro_loss(label_speed,E_mu,E_sigma)
        loss2 = (torch.abs(entire_out - label_time) / label).mean()

        tte_loss+=torch.abs(entire_out-label_time).sum()
        speed_loss+=torch.abs(label_speed-E_mu).sum()
        
        print('\r tte loss: %f speed loss %f' %(tte_loss.item(), speed_loss), end="")

        (loss1+loss2).mean().backward()
        optimizer.step()
        optimizer.zero_grad()
    print()
    print("Epoch:",epoch, tte_loss.item(),speed_loss.item())
    return tte_loss,speed_loss

In [8]:
def evaluate_traveltime(model, epoch,batch_size, test_data):
    model.eval()
    tte_loss,total_output,test_samples=0.0,0.0,[],0.0
    test_path,test_ratio,test_slots,test_orderid,test_label=test_data
    with torch.no_grad():
        for i in range(0, test_path.shape[0],batch_size):
            
            paths_temp,ratios_temp,time_temp=test_path[i:i+batch_size],test_ratio[i:i+batch_size],test_slots[i:i+batch_size]
            test_samples+=paths_temp.shape[0]
            entire_out,mu,sigma=model(road_idx,time_idx,time_temp,g2,u,lengths,paths_temp,edge_type)

            label_time=test_label[i:i+batch_size,0:1]
            total_output.append(entire_out)
            tte_loss+=torch.abs(entire_out-label_time).sum().item()
        tte_loss/=test_samples
        total_output=torch.cat(total_output)
        index=torch.unique(test_orderid)
        traj_output,label=torch.zeros(len(index),1),torch.zeros(len(index),1)
        for o, idx in enumerate(index):
            #max_lens=max(len(idx.reshape(-1)),max_lens)
            traj_output[o],label[o]=total_output[test_orderid==idx].sum(),test_label[test_orderid==idx].sum()
        traj_tte_loss=torch.abs(traj_output-label).mean()
       
        mse = compute_rmse(label, traj_output)
        mape = mean_absolute_percentage_error(label, traj_output)
        print("\n Epoch: %d, Path tte loss: %.4f, traj_tte_loss: %.4f" %( epoch,tte_loss/60,traj_tte_loss/60))
        return tte_loss,traj_tte_loss,mse,mape

#### Model Train 2 - Deep TTE

In [13]:
## Model
class Net(nn.Module):
    def __init__(self, kernel_size = 3, num_filter = 32, pooling_method = 'attention', num_final_fcs = 3, final_fc_size = 128, alpha = 0.3):
        super(Net, self).__init__()

        # parameter of attribute / spatio-temporal component
        self.kernel_size = kernel_size
        self.num_filter = num_filter
        self.pooling_method = pooling_method

        # parameter of multi-task learning component
        self.num_final_fcs = num_final_fcs
        self.final_fc_size = final_fc_size
        self.alpha = alpha

        self.build()
        self.init_weight()

    def init_weight(self):
        for name, param in self.named_parameters():
            if name.find('.bias') != -1:
                param.data.fill_(0)
            elif name.find('.weight') != -1:
                nn.init.xavier_uniform(param.data)

    def build(self):
        # attribute component
        self.attr_net = base.Attr.Net()

        # spatio-temporal component
        self.spatio_temporal = base.SpatioTemporal.Net(attr_size = self.attr_net.out_size(), \
                                                       kernel_size = self.kernel_size, \
                                                       num_filter = self.num_filter, \
                                                       pooling_method = self.pooling_method
        )

        self.entire_estimate = EntireEstimator(input_size =  self.spatio_temporal.out_size() + self.attr_net.out_size(), num_final_fcs = self.num_final_fcs, hidden_size = self.final_fc_size)

        self.local_estimate = LocalEstimator(input_size = self.spatio_temporal.out_size())


    def forward(self, attr, traj, config):
        attr_t = self.attr_net(attr)

        # sptm_s: hidden sequence (B * T * F); sptm_l: lens (list of int); sptm_t: merged tensor after attention/mean pooling
        sptm_s, sptm_l, sptm_t = self.spatio_temporal(traj, attr_t, config)

        entire_out = self.entire_estimate(attr_t, sptm_t)

        # sptm_s is a packed sequence (see pytorch doc for details), only used during the training
        if self.training:
            local_out = self.local_estimate(sptm_s[0])
            return entire_out, (local_out, sptm_l)
        else:
            return entire_out

    def eval_on_batch(self, attr, traj, config):
        if self.training:
            entire_out, (local_out, local_length) = self(attr, traj, config)
        else:
            entire_out = self(attr, traj, config)

        pred_dict, entire_loss = self.entire_estimate.eval_on_batch(entire_out, attr['time'], config['time_mean'], config['time_std'])

        if self.training:
            # get the mean/std of each local path
            mean, std = (self.kernel_size - 1) * config['time_gap_mean'], (self.kernel_size - 1) * config['time_gap_std']

            # get ground truth of each local path
            local_label = utils.get_local_seq(traj['time_gap'], self.kernel_size, mean, std)
            local_loss = self.local_estimate.eval_on_batch(local_out, local_length, local_label, mean, std)

            return pred_dict, (1 - self.alpha) * entire_loss + self.alpha * local_loss
        else:
            return pred_dict, entire_loss

In [12]:
EPS = 10

class EntireEstimator(nn.Module):
    def __init__(self, input_size, num_final_fcs, hidden_size = 128):
        super(EntireEstimator, self).__init__()

        self.input2hid = nn.Linear(input_size, hidden_size)

        self.residuals = nn.ModuleList()
        for i in range(num_final_fcs):
            self.residuals.append(nn.Linear(hidden_size, hidden_size))

        self.hid2out = nn.Linear(hidden_size, 1)

    def forward(self, attr_t, sptm_t):
        inputs = torch.cat((attr_t, sptm_t), dim = 1)

        hidden = F.leaky_relu(self.input2hid(inputs))

        for i in range(len(self.residuals)):
            residual = F.leaky_relu(self.residuals[i](hidden))
            hidden = hidden + residual

        out = self.hid2out(hidden)

        return out

    def eval_on_batch(self, pred, label, mean, std):
        label = label.view(-1, 1)

        label = label * std + mean
        pred = pred * std + mean

        loss = torch.abs(pred - label) / label

        return {'label': label, 'pred': pred}, loss.mean()

class LocalEstimator(nn.Module):
    def __init__(self, input_size):
        super(LocalEstimator, self).__init__()

        self.input2hid = nn.Linear(input_size, 64)
        self.hid2hid = nn.Linear(64, 32)
        self.hid2out = nn.Linear(32, 1)

    def forward(self, sptm_s):
        hidden = F.leaky_relu(self.input2hid(sptm_s))

        hidden = F.leaky_relu(self.hid2hid(hidden))

        out = self.hid2out(hidden)

        return out

    def eval_on_batch(self, pred, lens, label, mean, std):
        label = nn.utils.rnn.pack_padded_sequence(label, lens, batch_first = True)[0]
        label = label.view(-1, 1)

        label = label * std + mean
        pred = pred * std + mean

        loss = torch.abs(pred - label) / (label + EPS)

        return loss.mean()